In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [2]:
import glob
import os
import PIL
import time

from IPython import display

# Data

In [3]:
path_root = "C:/Users/Max/Documents/thesis_data"

In [4]:
batches = ImageDataGenerator().flow_from_directory(directory=path_root, color_mode = "rgb", 
                                                   target_size=(64,64), class_mode= "sparse", interpolation="bicubic", 
                                                   batch_size=40000)
imgs, labels = next(batches)
imgs.shape

Found 26548 images belonging to 11 classes.


(26548, 64, 64, 3)

In [5]:
batches.class_indices

{'BetterSurf': 0,
 'Eksor.A': 1,
 'Obfuscator.AFQ': 2,
 'Occamy.C': 3,
 'OnLineGames.CTB': 4,
 'Reveton.A': 5,
 'Sfone': 6,
 'VB.IL': 7,
 'Zbot': 8,
 'Zbot!CI': 9,
 'benign': 10}

In [6]:
num_classes = len(batches.class_indices)

In [7]:
testsize = 0.3
X_train, X_test, y_train, y_test = train_test_split(imgs, labels, test_size=testsize)

In [8]:
trainsize = len(X_train)
testsize = len(X_test)

print(f"Size of training data: {trainsize} | Shape of training data {X_train.shape}")
print(f"Size of training data: {testsize}  | Shape of training data {X_test.shape}")
print(f"Shape of training labels {y_train.shape}")
print(f"Shape of training labels {y_test.shape}")

Size of training data: 18583 | Shape of training data (18583, 64, 64, 3)
Size of training data: 7965  | Shape of training data (7965, 64, 64, 3)
Shape of training labels (18583,)
Shape of training labels (7965,)


# Preparing models

In [9]:
latent_dim = 128

generator = keras.Sequential(
    [
        keras.layers.InputLayer(input_shape=(latent_dim,)),
        layers.Dense(8 * 8 * 128),
        layers.Reshape((8, 8, 128)),
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid"),
    ],
    name="generator",
)
generator.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8192)              1056768   
_________________________________________________________________
reshape (Reshape)            (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 16, 16, 128)       262272    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 256)       524544    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 256)       0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 64, 64, 512)       20

In [10]:
discriminator = keras.Sequential(
    [
        
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same", input_shape=(64, 64, 3)),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Flatten(),
        layers.Dropout(0.2),
        layers.Dense(1, activation="sigmoid"),
    ],
    name="discriminator",
)
discriminator.summary()

Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        3136      
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       131200    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)         262272    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 8, 8, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)            

In [11]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [12]:
checkpoint_dir = 'C:/Users/Max/Documents/gan_checkpoint'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [13]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

Now, we transfer the weights of discriminator except the last layer to a new model, add a dense layer with 128 units, and add another dense layer with 10 units and softmax activation.

In [28]:
disc = checkpoint.discriminator
    
new_model = Sequential()
for i in range(len(disc.layers) - 1):
    new_model.add(disc.layers[i])

for layer in new_model.layers:
    layers.trainable = False

new_model.add(Dense(128, activation = "relu"))
new_model.add(Dense(num_classes, activation = "softmax"))
new_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        3136      
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       131200    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)         262272    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 8, 8, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)             

In [29]:
# view the initialized weights and bias of the second last dense layer; weights are uniformly randomly generated 
# and biases are all zeroes by default
new_model.layers[-2].weights

[<tf.Variable 'dense_6/kernel:0' shape=(8192, 128) dtype=float32, numpy=
 array([[ 0.01305944,  0.01706236,  0.01150295, ..., -0.01543331,
         -0.00646758, -0.01715975],
        [-0.00397384, -0.0240164 , -0.00508745, ...,  0.0075558 ,
          0.0237201 , -0.0148007 ],
        [ 0.01351043,  0.00886332, -0.01109991, ..., -0.02456664,
         -0.0115023 , -0.00054914],
        ...,
        [ 0.00644967,  0.00618113, -0.02109783, ...,  0.02318005,
         -0.01724467,  0.01905178],
        [-0.0176617 , -0.01398314, -0.00472712, ..., -0.01428101,
          0.0241043 , -0.00593137],
        [-0.01580074,  0.00928331,  0.02645567, ...,  0.00711003,
          0.01465366,  0.01382229]], dtype=float32)>,
 <tf.Variable 'dense_6/bias:0' shape=(128,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [30]:
# train_images = X_train.reshape(( 64, 64, 1))
# train_images = (train_images.astype('float32') - 127.5) / 127.5

# test_images = X_test.reshape(( 64, 64, 1))
# test_images = (test_images.astype('float32') - 127.5) / 127.5

train_images = X_train.reshape((trainsize, 64,64,3))
train_images = train_images.astype('float32') / 255 # Was *255
train_images.shape

(18583, 64, 64, 3)

In [31]:
             
test_images = X_test.reshape((testsize, 64,64,3))
test_images = test_images.astype('float32') / 255 # Was *255

In [32]:
train_labels = to_categorical(y_train)
test_labels = to_categorical(y_test)

print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)

(18583, 64, 64, 3) (18583, 11) (7965, 64, 64, 3) (7965, 11)


In [33]:
new_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

history1 = new_model.fit(train_images, train_labels, batch_size=100, epochs=7,
                        validation_data=(test_images, test_labels))

Epoch 1/7
186/186 [==============================] - 4s 19ms/step - loss: nan - accuracy: 0.0919 - val_loss: nan - val_accuracy: 0.0913
Epoch 2/7
186/186 [==============================] - 3s 17ms/step - loss: nan - accuracy: 0.0919 - val_loss: nan - val_accuracy: 0.0913
Epoch 3/7
186/186 [==============================] - 3s 17ms/step - loss: nan - accuracy: 0.0919 - val_loss: nan - val_accuracy: 0.0913
Epoch 4/7
186/186 [==============================] - 3s 17ms/step - loss: nan - accuracy: 0.0919 - val_loss: nan - val_accuracy: 0.0913
Epoch 5/7
186/186 [==============================] - 3s 18ms/step - loss: nan - accuracy: 0.0919 - val_loss: nan - val_accuracy: 0.0913
Epoch 6/7
186/186 [==============================] - 3s 17ms/step - loss: nan - accuracy: 0.0919 - val_loss: nan - val_accuracy: 0.0913
Epoch 7/7
186/186 [==============================] - 3s 18ms/step - loss: nan - accuracy: 0.0919 - val_loss: nan - val_accuracy: 0.0913


In [34]:
# unfreeze all layers 
for layer in new_model.layers:
    layer.trainable = True

#optimizer=new_model.optimizer
#optimizer.learning_rate=0.005
    
new_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

history2 = new_model.fit(train_images, train_labels, batch_size=100, epochs=8,
                        validation_data=(test_images, test_labels)) 


Epoch 1/8
186/186 [==============================] - 4s 18ms/step - loss: nan - accuracy: 0.0919 - val_loss: nan - val_accuracy: 0.0913
Epoch 2/8
186/186 [==============================] - 3s 17ms/step - loss: nan - accuracy: 0.0919 - val_loss: nan - val_accuracy: 0.0913
Epoch 3/8
186/186 [==============================] - 3s 18ms/step - loss: nan - accuracy: 0.0919 - val_loss: nan - val_accuracy: 0.0913
Epoch 4/8
186/186 [==============================] - 3s 18ms/step - loss: nan - accuracy: 0.0919 - val_loss: nan - val_accuracy: 0.0913
Epoch 5/8
186/186 [==============================] - 3s 17ms/step - loss: nan - accuracy: 0.0919 - val_loss: nan - val_accuracy: 0.0913
Epoch 6/8
186/186 [==============================] - 3s 17ms/step - loss: nan - accuracy: 0.0919 - val_loss: nan - val_accuracy: 0.0913
Epoch 7/8
186/186 [==============================] - 3s 17ms/step - loss: nan - accuracy: 0.0919 - val_loss: nan - val_accuracy: 0.0913
Epoch 8/8
186/186 [=============================

In [ ]:
accuracy = history1.history['accuracy'] + history2.history['accuracy']
val_accuracy = history1.history['val_accuracy'] + history2.history['val_accuracy']

plt.figure()
plt.plot(accuracy, label='accuracy')
plt.plot(val_accuracy, label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.3, 1])
plt.legend(loc='lower right')
plt.show()

We can see that, using the tuned discriminator model, both training accuracy and testing are approximately 0.94 after 15 epochs.

In [ ]:
# demonstration: predict the ith test digit
i = np.random.randint(0, trainsize)

# show the actual ith digit
print('actual label:', np.argmax(test_labels[i]))
plt.figure()
plt.imshow(test_images[i,:,:,0], cmap='gray')
plt.show()

# predict
prediction = new_model.predict(test_images[i].reshape(1,28,28,1))

# get probability distribution and classification of the test digit
print(prediction)
print('prediction:', np.argmax(prediction))

# draw the barplot
plt.figure()
plt.bar(np.arange(0,10).astype('str'), prediction[0,:])
plt.show()

**Get Confusion Matrix**

In [ ]:
pred_labels = new_model.predict(test_images).argmax(axis=1)
true_labels = test_labels.argmax(axis=1)

In [ ]:
confusion_mat = tf.math.confusion_matrix(labels=true_labels, predictions=pred_labels).numpy()
confusion_mat

In [ ]:
import pandas as pd
import seaborn as sns

confusion_mat_norm = np.around(confusion_mat.astype('float') / np.atleast_2d(confusion_mat.sum(axis=1)).T, decimals=2)

classes = np.arange(0,10).astype('str')
confusion_mat_df = pd.DataFrame(confusion_mat_norm,
                                index = classes, 
                                columns = classes)

figure = plt.figure()
sns.heatmap(confusion_mat_df, annot=True, cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

**Generate new digits and perform classification on them using our model**

In [ ]:
noi = tf.random.normal([1, 100])
sample = checkpoint.generator(noi, training=False)
fig = plt.figure()
plt.imshow(sample[0, :, :, 0], cmap='gray')

In [ ]:
new_sample = sample[0, :, :, 0].numpy().reshape(1,28,28,1)
new_prediction = new_model.predict(new_sample)

# get probability distribution and classification of the test digit
print(new_prediction)
print('prediction:', np.argmax(new_prediction))

# draw the barplot
plt.figure()
plt.bar(np.arange(0,10).astype('str'), new_prediction[0,:])
plt.show()

In [ ]:
# we can save the numpy array of appropriately generated digits as .npy file,
# which could be used for further training
np.save('generated_numpy.npy', sample[0, :, :, 0].numpy())

In [ ]:
# upload the saved numpy array
arr = np.load('generated_numpy.npy')
arr